In [5]:
import pandas as pd
import numpy as np
from cmath import inf
from geopy.distance import geodesic
from tqdm import tqdm
import time

In [7]:
# read data
data = pd.read_csv('/Users/pranshusavani/JupyterFiles/CS 541/hw2/nyc_taxi_trips/nyc_taxi_data.csv')
data = data[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','trip_distance']]

# remove 0 values
data = data[(data[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','trip_distance']] != 0).all(axis=1)]
data = round(data,4)

# get all lattitudes and longitudes in one column
nodes = data[['pickup_longitude','pickup_latitude']]
nodes = nodes.rename(columns={'pickup_longitude':'long','pickup_latitude':'lat'})
nodes2 = data[['dropoff_longitude','dropoff_latitude']]
nodes2 = nodes2.rename(columns={'dropoff_longitude':'long','dropoff_latitude':'lat'})
nodes = nodes.append(nodes2,ignore_index=True)

# create nodes.csv
nodes = nodes.groupby(['long','lat']).size().reset_index().rename(columns={0:'nodeid'})
nodes['nodeid'] = nodes.index.values
nodes.to_csv('/Users/pranshusavani/JupyterFiles/CS 541/hw2/nyc_taxi_trips/nodes.csv')

# create edges.csv
edges = pd.DataFrame(np.zeros((len(data),3)),columns=['nodeid1','nodeid2','cost'])
for i,row in data.iterrows():
    edges['nodeid1'][i] = nodes[(nodes['long']==row['pickup_longitude']) & (nodes['lat']==row['pickup_latitude'])]['nodeid']
    edges['nodeid2'][i] = nodes[(nodes['long']==row['dropoff_longitude']) & (nodes['lat']==row['dropoff_latitude'])]['nodeid']
    edges['cost'][i] = row['trip_distance']
edges = edges.drop_duplicates(subset=['nodeid1', 'nodeid2'], keep='first')
edges.to_csv('/Users/pranshusavani/JupyterFiles/CS 541/hw2/nyc_taxi_trips/edges.csv')

In [8]:
# Class for node
class Node:
    def __init__(self, val, coordinates, next=None):
        self.lat = coordinates[0]
        self.long = coordinates[1]
        self.val = val
        self.cost = inf
        self.parent = None
        if next is None:
            self.next = []
        else:
            self.next = next
        self.heuristic_value = -1

    def __gt__(self, other):
        if isinstance(other, Node):
            if self.heuristic_value > other.heuristic_value:
                return True
            if self.heuristic_value < other.heuristic_value:
                return False
            return self.val > other.val

# class for graph
class Graph:
    def __init__(self):
        self.nodes = []

    def add_edge(self, val1, val2, c=1):
        self.get_node(val1).next.append((self.get_node(val2),c))
        self.get_node(val2).next.append((self.get_node(val1),c))

    
    def get_node(self, val):
        for n in self.nodes:
            if n.val == val:
                return n
        return None

In [9]:
# class for search algorithms Uniform Cost Search and A* Search
class MySearch:
  def __init__(self, graph, start_position, end_position,search_choice='ucs'):
    self.search_choice = search_choice
    self.start = graph.get_node(start_position)
    self.end = graph.get_node(end_position)
    self.graph = graph
    self.frontier = []
    self.visited = []
  
  def get_total_cost(self, path):
    total_cost = 0
    for i in range(len(path) - 1):
      child = self.graph.get_node(path[i])
      parent = self.graph.get_node(path[i+1])
      for n in child.next:
        if n[0] == parent:
          total_cost += n[1]
    return total_cost
  
  def flag_visited(self):
    self.frontier.sort()
    node = self.frontier.pop(0)
    self.visited.append(node)
    return node

  def straight_line_distance(self, node1, node2):
    return geodesic((node1.lat,node1.long),(node2.lat,node2.long)).miles

  def get_cost(self, parent, child):
    for n in parent.next:
      if n[0] == child:
        cost = parent.cost + n[1]
        if cost < child.cost:
          child.parent = parent
          return cost
        return child.cost
  
  def get_path(self, n):
    path = [n.val]
    node = n.parent
    while True:
      path.append(node.val)
      if node.parent is None:
        break
      node = node.parent
    return path

  def check_frontier(self):
    return len(self.frontier) == 0
    
  def insert(self, list_category, node):
    if list_category == 'frontier':
      self.frontier.append(node)
    else:
      self.visited.append(node)

  def get_prev_node(self, val):
    for node in self.frontier:
      if node.val == val:
        return node
    return None 
  
  def get_heuristic_value(self, node1, node2, end):
    if self.search_choice=='ucs':
      return self.get_cost(node1, node2)
    elif self.search_choice=='astar':
      return self.get_cost(node1, node2) + self.straight_line_distance(node2, end)
    
  def search(self):
    self.start.cost = 0
    self.start.heuristic_value = self.straight_line_distance(self.start, self.end)
    self.frontier.append(self.start)
    while True:
      if self.check_frontier():
        print('no path exists!')
        break
      current_node = self.flag_visited()
      if current_node == self.end:
        path = self.get_path(current_node)
        total_cost = self.get_total_cost(path)
        path.reverse()
        return total_cost
      new_nodes = []
      for c in current_node.next:
        new_nodes.append(c[0])
      if len(new_nodes) > 0:
        for new_node in new_nodes:
          new_node.heuristic_value = self.get_heuristic_value(current_node, new_node, self.end)
          if new_node not in self.visited and new_node not in self.frontier:
            new_node.parent = current_node
            self.insert("frontier", new_node)
          elif new_node in self.frontier and new_node.parent != current_node:
            old_node = self.get_prev_node(new_node.val)
            if new_node.heuristic_value < old_node.heuristic_value:
              new_node.parent = current_node
              self.insert_to_opened(new_node)

In [10]:
# Create graph
graph = Graph()
# Add vertices
for idx,node in tqdm(nodes.iterrows()):
  n = Node(str(node['nodeid']),(node['lat'],node['long']))
  graph.nodes.append(n)

# Add edges
for idx,edge in tqdm(edges.iterrows()):
  graph.add_edge(str(edge['nodeid1']), str(edge['nodeid2']), edge['cost'])


start_t = time.time()
# execute Uniform Cost Search
ucs = MySearch(graph, '103211.0', '95564.0','ucs')
ucs_cost = ucs.search()
print('Total Cost Uniform Cost Search: {:.2f}'.format(ucs_cost))
print('Time Taken Uniform Cost Search: {:.2f} mins'.format((time.time()-start_t)/60))


start_t = time.time()
# execute A* Search
astar = MySearch(graph, '103211.0', '95564.0','astar')
astar_cost = astar.search()
print('Total Cost A* Search: {:.2f}'.format(astar_cost))
print('Time Taken A* Search: {:.2f} mins'.format((time.time()-start_t)/60))



145274it [00:02, 53302.95it/s]
167931it [27:03, 103.43it/s]


Total Cost Uniform Cost Search: 7.3100000000000005
Time Taken Uniform Cost Search: 1782.1017026901245
Total Cost A* Search: 7.3100000000000005
Time Taken A* Search: 1815.0189218521118
